# 컵퓨터 비전 학습 경진 대회
---
- `Private Score : 0.86733`
- `Rank : 12`
- 구글 Colab을 통해 진행하였고, 팀원 각자 모델을 돌려 마지막에 앙상블하여 최종 결과 도출.
- 처음 해보는 컴퓨터 비전 경진대회라 많은 어려움이 있었지만 팀원들의 도움으로 12위라는 성적을 기록 할 수 있었다고 생각합니다. . 


데이터 불러오기.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from google.colab import output

# data_2.zip을 현재 디렉터리에 압축해제
!unzip "data_2.zip"

In [ ]:
from google.colab import output
# 현재 디렉터리에 dirty_mnist라는 폴더 생성
!mkdir "./dirty_mnist"
#dirty_mnist.zip라는 zip파일을 dirty_mnist라는 폴더에 압축 풀기
!unzip "dirty_mnist_2nd.zip" -d "./dirty_mnist/"
# 현재 디렉터리에 test_dirty_mnist라는 폴더 생성
!mkdir "./test_dirty_mnist"
#test_dirty_mnist.zip라는 zip파일을 test_dirty_mnist라는 폴더에 압축 풀기
!unzip "test_dirty_mnist_2nd.zip" -d "./test_dirty_mnist/"
# 출력 결과 지우기
output.clear()

## 라이브러리 로딩

In [ ]:
!pip install efficientnet_pytorch
!pip install timm

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
import imutils
import zipfile
import os
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from google.colab import output
import random
import time

from sklearn.model_selection import KFold



device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # 디바이스 설정


## 시드 고정 

In [ ]:
# https://dacon.io/competitions/official/235697/codeshare/2363?page=1&dtype=recent&ptype=pub

def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  
    torch.backends.cudnn.deterministic = True  
    torch.backends.cudnn.benchmark = True  
seed_everything()

## 커스텀 데이터셋 만들기

In [ ]:
dirty_mnist_answer = pd.read_csv("dirty_mnist_2nd_answer.csv")
# dirty_mnist라는 디렉터리 속에 들어있는 파일들의 이름을 
# namelist라는 변수에 저장
namelist = os.listdir('./dirty_mnist/')

# numpy를 tensor로 변환하는 ToTensor 정의
class ToTensor(object):
    """numpy array를 tensor(torch)로 변환합니다."""
    def __call__(self, sample):
        image, label = sample['image'], sample['label']
        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        image = image.transpose((2, 0, 1))
        return {'image': torch.FloatTensor(image),
                'label': torch.FloatTensor(label)}
# to_tensor 선언

to_tensor = transforms.Compose([
                      ToTensor() 
])
class DatasetMNIST(torch.utils.data.Dataset):
    def __init__(self,
                 dir_path,
                 meta_df,
                 transforms=to_tensor,#미리 선언한 to_tensor를 transforms로 받음
                 augmentations=None):
        
        self.dir_path = dir_path # 데이터의 이미지가 저장된 디렉터리 경로
        self.meta_df = meta_df # 데이터의 인덱스와 정답지가 들어있는 DataFrame

        self.transforms = transforms# Transform
        self.augmentations = augmentations # Augmentation
        
    def __len__(self):
        return len(self.meta_df)
    
    def __getitem__(self, index):
        # 폴더 경로 + 이미지 이름 + .png => 파일의 경로
        # 참고) "12".zfill(5) => 000012
        #       "146".zfill(5) => 000145
        # cv2.IMREAD_GRAYSCALE : png파일을 채널이 1개인 GRAYSCALE로 읽음
        image = cv2.imread(self.dir_path +\
                           str(self.meta_df.iloc[index,0]).zfill(5) + '.png',
                           cv2.IMREAD_GRAYSCALE)
        # 0 ~ 255의 값을 갖고 크기가 (256,256)인 numpy array를
        # 0 ~ 1 사이의 실수를 갖고 크기가 (256,256,1)인 numpy array로 변환
        image = (image/255).astype('float32')[..., np.newaxis]

        # 정답 numpy array생성(존재하면 1 없으면 0)
        label = self.meta_df.iloc[index, 1:].values.astype('float')
        sample = {'image': image, 'label': label}

        # transform 적용
        # numpy to tensor
        if self.transforms:
            sample = self.transforms(sample)
        if self.augmentations:
            sample['image'] = self.augmentations(sample['image'])
        # sample 반환
        return sample

## pre_trained 모델 로딩(effnet-b7)
---
- 최종 모델로 efficientnet 을 선정한 이유는 성능이 가장 좋았기 떄문입니다.
- [참고 깃허브](https://github.com/lukemelas/EfficientNet-PyTorch)
- `regnet_32` : public_score (0.8314) , `Resnet` : 0.532, `Mobilenet_v3` : 0.7558 , `effnet_b7` : 0.865 

In [10]:
from model import effnet_b7
from model import regnet
from model import mobilenet_v3

#model = regnet.MultiLabelRegnet()
#model = mobilenet_v3.moblienet_v3()
# 제일 성능이 좋은 effnet 사용.
model = effnet_b7.MultiLabeleffnet()

Loaded pretrained weights for efficientnet-b7


## K-fold 교차검증을 통한 학습진행
---
- k = 3

In [ ]:
kf = KFold(n_splits=3, shuffle=True, random_state=42)
folds=[]
for train_idx, valid_idx in kf.split(dirty_mnist_answer):
    folds.append((train_idx, valid_idx))    

### 학습파라미터
- Epochs = 30
- batch_size = 16
- Optimizer = Adam (lr = 0.001)
- lr_scheduler = StepLR(optimizer,step_size = 5,gamma = 0.9)


In [ ]:
epochs=30
batch_size=16   
model = MultiLabeleffnet()
model.to(device)# gpu에 모델 할당
optimizer = torch.optim.Adam(model.parameters(),lr = 0.001)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                            step_size = 5,
                                            gamma = 0.9)
criterion = torch.nn.BCELoss()

Loaded pretrained weights for efficientnet-b7


In [ ]:
for fold in range(3):

    # dirty_mnist_answer에서 train_idx와 val_idx를 생성
    best_models = [] # 폴드별로 가장 validation acc가 높은 모델 저장
    train_idx = folds[fold][0]
    valid_idx = folds[fold][1]

    # cuda cache 초기화
    torch.cuda.empty_cache()

    #train fold, validation fold 분할
    train_answer = dirty_mnist_answer.iloc[train_idx]
    test_answer  = dirty_mnist_answer.iloc[valid_idx]

    # augmentations 적용 
    train_augmentations = transforms.Compose([
            transforms.ToPILImage(),        
            transforms.RandomHorizontalFlip(p=0.6),
            transforms.RandomVerticalFlip(0.6),
            transforms.RandomRotation(40),
            transforms.ToTensor(),
            
            ])
    valid_augmentations = transforms.Compose([
        transforms.ToPILImage(),
        transforms.ToTensor(),
        
        ])
    train_answer = dirty_mnist_answer.iloc[train_idx]
    test_answer  = dirty_mnist_answer.iloc[valid_idx]

    # Data Loader
    train_dataset = DatasetMNIST("dirty_mnist/", train_answer, augmentations=train_augmentations)
    valid_dataset = DatasetMNIST("dirty_mnist/", test_answer, augmentations= valid_augmentations)

    train_data_loader = DataLoader(
        train_dataset,
        batch_size = batch_size,
        shuffle = True,
        num_workers = 3
    )
    valid_data_loader = DataLoader(
        valid_dataset,
        batch_size = batch_size,
        shuffle = False,
        num_workers = 3
    )




    # 훈련 시작
    valid_acc_max = 0
    for epoch in range(epochs):
        # 1개 epoch 훈련
        train_acc_list = []
        with tqdm(train_data_loader,#train_data_loader를 iterative하게 반환
                total=train_data_loader.__len__(), # train_data_loader의 크기
                unit="batch") as train_bar:# 한번 반환하는 smaple의 단위는 "batch"
            for sample in train_bar:
                train_bar.set_description(f"Train Epoch {epoch}")
                # 갱신할 변수들에 대한 모든 변화도를 0으로 초기화
                # 참고)https://tutorials.pytorch.kr/beginner/pytorch_with_examples.html
                optimizer.zero_grad()
                images, labels = sample['image'], sample['label']
                # tensor를 gpu에 올리기 
                images = images.to(device)
                labels = labels.to(device)

                # 모델의 dropoupt, batchnormalization를 train 모드로 설정
                model.train()
                # .forward()에서 중간 노드의 gradient를 계산
                with torch.set_grad_enabled(True):
                    # 모델 예측
                    probs  = model(images)
                    # loss 계산
                    loss = criterion(probs, labels)

                    # 중간 노드의 gradient로
                    # backpropagation을 적용하여
                    # gradient 계산
                    loss.backward()

                    # weight 갱신
                    optimizer.step()

                    # train accuracy 계산
                    probs  = probs.cpu().detach().numpy()
                    labels = labels.cpu().detach().numpy()
                    preds = probs > 0.5
                    batch_acc = (labels == preds).mean()
                    train_acc_list.append(batch_acc)
                    train_acc = np.mean(train_acc_list)

                # 현재 progress bar에 현재 미니배치의 loss 결과 출력
                train_bar.set_postfix(train_loss= loss.item(),
                                        train_acc = train_acc)
                


        # 1개 epoch학습 후 Validation 점수 계산
        valid_acc_list = []
        
        with tqdm(valid_data_loader,
                total=valid_data_loader.__len__(),
                unit="batch") as valid_bar:
            for sample in valid_bar:
                valid_bar.set_description(f"Valid Epoch {epoch}")
                optimizer.zero_grad()
                images, labels = sample['image'], sample['label']
                images = images.to(device)
                labels = labels.to(device)

                # 모델의 dropoupt, batchnormalization를 eval모드로 설정
                model.eval()
                # .forward()에서 중간 노드의 gradient를 계산
                with torch.no_grad():
                    # validation loss만을 계산
                    probs  = model(images)
                    valid_loss = criterion(probs, labels)

                    # train accuracy 계산
                    probs  = probs.cpu().detach().numpy()
                    labels = labels.cpu().detach().numpy()
                    preds = probs > 0.5
                    batch_acc = (labels == preds).mean()
                    valid_acc_list.append(batch_acc)

                valid_acc = np.mean(valid_acc_list)
                valid_bar.set_postfix(valid_loss = valid_loss.item(),
                                        valid_acc = valid_acc)
            
        # Learning rate 조절
        lr_scheduler.step()

        # 모델 저장
        if valid_acc_max < valid_acc:
            valid_acc_max = valid_acc
            best_model = model
            MODEL = "effnet_b7"
            # 모델을 저장할 구글 드라이브 경로
            path = "drive/MyDrive/effnet"
        
            torch.save(best_model, f'{path}_{MODEL}_{valid_loss.item():2.4f}_epoch_{epoch}_fold_{fold}.pth')

        # 폴드별로 가장 좋은 모델 저장
        best_models.append(best_model)
        

## 테스트를위한 모델 불러오기
---
- 각 fold별로 loss 값을 확인하여 저장된 모델을 불러왔습니다.
- 저의 경우 총 8개의 모델을 불러왔습니다.

In [ ]:


model1 = torch.load('/content/drive/MyDrive/effnet/_effnet_b7_0.1705_epoch_18_fold_0.pth')
model2 = torch.load('/content/drive/MyDrive/effnet/_effnet_b7_0.1794_epoch_20_fold_0.pth')
model3 = torch.load('/content/drive/MyDrive/effnet/_effnet_b7_0.1827_epoch_16_fold_0.pth')
model4 = torch.load('/content/drive/MyDrive/effnet/effnet_effnet_b7_0.1770_epoch_24_fold_1.pth')
model5 = torch.load('/content/drive/MyDrive/effnet/effnet_effnet_b7_0.1917_epoch_24_fold_1.pth')
model6 = torch.load('/content/drive/MyDrive/effnet/effnet_effnet_b7_0.1957_epoch_23_fold_1.pth')
model7 = torch.load('/content/drive/MyDrive/effnet/effnet_effnet_b7_0.2029_epoch_17_fold_2.pth')
model8 = torch.load('/content/drive/MyDrive/effnet/effnet_effnet_b7_0.2081_epoch_18_fold_2.pth')


best_models = []
best_models.append(model1)
best_models.append(model2)
best_models.append(model3)
best_models.append(model4)
best_models.append(model5)
best_models.append(model6)
best_models.append(model7)
best_models.append(model8)

## 모델 테스트

In [ ]:
#test Dataset 정의
sample_submission = pd.read_csv("sample_submission.csv")
test_dataset = DatasetMNIST("test_dirty_mnist/", sample_submission)
batch_size = 128
test_data_loader = DataLoader(
    test_dataset,
    batch_size = batch_size,
    shuffle = False,
    num_workers = 3,
    drop_last = False
)

In [ ]:

probs_list = []
# 배치 단위로 추론
prediction_df = pd.read_csv("sample_submission.csv")

# 5개의 fold마다 가장 좋은 모델을 이용하여 예측
for model in best_models:
    # 0으로 채워진 array 생성
    probs_array = np.zeros([prediction_df.shape[0],
                                 prediction_df.shape[1] -1])
    
    for idx, sample in enumerate(test_data_loader):
        with torch.no_grad():
            # 추론
            model.eval()
            images = sample['image']
            images = images.to(device)
            probs  = model(images)
            probs = probs.cpu().detach().numpy()
            

            # 예측 결과를 
            # prediction_array에 입력
            batch_index = batch_size * idx
            probs_array[batch_index: batch_index + images.shape[0],:]\
                         = probs
    probs_list.append(probs_array[...,np.newaxis])                         
    

## 모델 앙상블 
---
3명의 팀원이 각자 모델을 학습하여 최종적으로 3개의 모델을 앙상블 하려고 하였으나,  
한개의 모델의 성능이 다소 낮았기에 2개의 모델만을 항상블하여 결과 추출

In [ ]:
# porb를 통해 평균값을 구해줌(hyup)
porb_list = np.concatenate(probs_list, axis = 2)
hyup = porb_list.mean(axis = 2)

In [ ]:
probs_list = []

# 다소 낮은 성능의 모델이라 앙상블 하지 않음
#hk = np.load('/content/drive/MyDrive/effnet/hkl_probs_mean_final.npy')
#probs_list.append(hk[...,np.newaxis])

jw = np.load('/content/drive/MyDrive/effnet/jw_probs_mean.npy')
probs_list.append(jw[...,np.newaxis])

probs_list.append(hyup[...,np.newaxis])


# axis = 2를 기준으로 평균
probs_array = np.concatenate(probs_list, axis = 2)
probs_mean = probs_array.mean(axis = 2)

# 평균 값이 0.5보다 클 경우 1 작으면 0
probs_mean = (probs_mean > 0.5) * 1

# 제출파일 생성
sample_submission = pd.read_csv("sample_submission.csv")
sample_submission.iloc[:,1:] = probs_mean
sample_submission.to_csv("probs_final(2).csv", index = False)
sample_submission

,index,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z
0,50000,1,0,1,0,1,1,0,1,1,0,0,0,0,1,0,1,0,0,0,1,1,1,0,1,0,1
1,50001,0,1,1,0,1,0,0,0,1,1,1,1,0,0,1,1,0,1,0,0,1,1,0,0,0,0
2,50002,0,0,0,1,1,0,1,0,1,0,1,1,1,0,0,1,0,1,1,0,1,0,1,0,0,1
3,50003,1,1,0,0,0,1,1,0,1,0,1,1,1,0,0,1,0,0,0,1,0,1,0,1,0,1
4,50004,0,0,1,0,1,1,0,0,0,0,0,1,1,0,1,0,1,0,1,1,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,54995,0,1,1,0,0,0,0,1,0,0,1,1,1,0,0,0,0,1,0,1,1,1,1,0,1,0
4996,54996,1,1,1,0,1,0,0,0,1,1,0,0,0,0,1,1,0,0,1,0,0,0,0,1,0,1
4997,54997,1,0,0,1,0,1,0,0,0,1,1,0,1,0,1,1,0,0,0,1,1,1,1,0,0,1
4998,54998,0,0,1,0,0,0,1,1,1,1,1,1,0,0,0,0,1,1,0,1,0,0,1,0,0,1
